<a href="https://colab.research.google.com/github/AlafateABULIMITI/NLP_Rush/blob/master/Web_mining_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%shell
wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1sJzVrwDZyWvmKFGS2YH1Xg4MyVtbSZf3' -O data.zip
unzip data

--2019-12-12 12:15:42--  https://docs.google.com/uc?export=download&id=1sJzVrwDZyWvmKFGS2YH1Xg4MyVtbSZf3
Resolving docs.google.com (docs.google.com)... 74.125.195.102, 74.125.195.113, 74.125.195.139, ...
Connecting to docs.google.com (docs.google.com)|74.125.195.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0g-ac-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/4lehja9gg8j91vhnd8ub5erpaa0rejjd/1576152000000/01123303726322883128/*/1sJzVrwDZyWvmKFGS2YH1Xg4MyVtbSZf3?e=download [following]
--2019-12-12 12:15:42--  https://doc-0g-ac-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/4lehja9gg8j91vhnd8ub5erpaa0rejjd/1576152000000/01123303726322883128/*/1sJzVrwDZyWvmKFGS2YH1Xg4MyVtbSZf3?e=download
Resolving doc-0g-ac-docs.googleusercontent.com (doc-0g-ac-docs.googleusercontent.com)... 74.125.142.132, 2607:f8b0:400e:c08::84
Connecting to doc-0g-ac-docs.googleusercontent.com (doc-0g-ac-d

In [4]:
##################################
# Step 1
# Import log file as a dataframe
##################################
verbose = True

import re
import pandas as pd

if verbose:
    print("Parsing log file into Pandas dataframe")

IP = []
ID = []
DATE = []
URL = []
STATUS = []
SENT = []
REF = []
AGENT = []

path = "../content/logs/bourges.txt"
pattern = re.compile(
    '^([^ ]+) ([^ ]+) [^ ]+ \[([^\]]+)\] "([^"]+)" ([0-9]{3}) ([^ ]+) "([^"]+)" "([^"]+)"$'
)
fields = ["IP", "ID", "DATE", "URL", "STATUS", "SENT", "REF", "AGENT"]

with open(path, "r") as f:
    for line in f:
        g = re.match(pattern, line).groups()
        IP.append(g[0])
        ID.append(g[1])
        DATE.append(g[2])
        URL.append(g[3])
        STATUS.append(g[4])
        SENT.append(g[5])
        REF.append(g[6])
        AGENT.append(g[7])
cols = zip(IP, ID, DATE, URL, STATUS, SENT, REF, AGENT)  # NIL,
df = pd.DataFrame(cols, columns=fields)
df.head()

Parsing log file into Pandas dataframe


,IP,ID,DATE,URL,STATUS,SENT,REF,AGENT
0,crawler10.googlebot.com,-,05/Sep/2004:04:26:24 +0200,GET /html/rhinolophides.html HTTP/1.0,304,-,-,Googlebot/2.1 (+http://www.google.com/bot.html)
1,crawler13.googlebot.com,-,05/Sep/2004:04:30:00 +0200,GET /html/jeunesse.htm HTTP/1.0,304,-,-,Googlebot/2.1 (+http://www.google.com/bot.html)
2,crawler14.googlebot.com,-,05/Sep/2004:04:31:22 +0200,GET / HTTP/1.0,304,-,-,Googlebot/2.1 (+http://www.google.com/bot.html)
3,crawler11.googlebot.com,-,05/Sep/2004:04:32:20 +0200,GET /anglais/html/index_studies.htm HTTP/1.0,200,700,-,Googlebot/2.1 (+http://www.google.com/bot.html)
4,crawler14.googlebot.com,-,05/Sep/2004:04:47:23 +0200,GET /html/non_exclu.htm HTTP/1.0,304,-,-,Googlebot/2.1 (+http://www.google.com/bot.html)


In [5]:
##################################
# Step 2
# Convert date string to datetime format
# Remove get/post and HTTP/ from URLs
##################################
if verbose:
    print("Parsing date string into datetime")
# Changing to datetime
import datetime as dt
import pytz

timezone_nw = pytz.timezone("Etc/GMT-2")

dateformat = "%d/%b/%Y:%H:%M:%S"
corrected_date = []
for date_time_str in df["DATE"]:
    date_short = date_time_str.split(" ")[0]
    datetime = dt.datetime.strptime(date_short, dateformat)  # import from date format
    final_date = datetime.astimezone(timezone_nw)  # change the timezone to +2:00
    corrected_date.append(final_date)  # build a list of datetime
df["DATE"] = corrected_date

urlformat = "^(GET|POST|HEAD) ([^ ]+) (HTTP.*)$"
urlpattern = re.compile(urlformat)
newurls = []
for url_str in df["URL"]:
    if re.match(urlpattern, url_str):
        g = re.match(urlpattern, url_str).groups()
        newurls.append(g[1].lower())
    else:
        newurls.append(url_str.lower()[0:128])
df["URL"] = newurls
print(df.head(5))


Parsing date string into datetime
                        IP  ...                                            AGENT
0  crawler10.googlebot.com  ...  Googlebot/2.1 (+http://www.google.com/bot.html)
1  crawler13.googlebot.com  ...  Googlebot/2.1 (+http://www.google.com/bot.html)
2  crawler14.googlebot.com  ...  Googlebot/2.1 (+http://www.google.com/bot.html)
3  crawler11.googlebot.com  ...  Googlebot/2.1 (+http://www.google.com/bot.html)
4  crawler14.googlebot.com  ...  Googlebot/2.1 (+http://www.google.com/bot.html)

[5 rows x 8 columns]


In [6]:
##################################
# Step 3
# Build sessions and urls list
##################################
if verbose:
    print("Building sessions and urls dictionary")

urls = []  # URLS list: assign a unique ID to each URL
sessions = {}  # key: id_session, value = {id_user, [url], [timestamp]}
tmp = {}  #
nb_sessions = 0
for index, entry in df.iterrows():
    # step 3.1. Retrieving / encoding the URL
    if not (entry["URL"] in urls):
        urls.append(entry["URL"])
    # step 3.2. Building a new session or extending an existing session
    # Current on-going sessions are stored in tmp
    if entry["IP"] in tmp:  # scanning keys of tmp dict to find the same user
        u, t = tmp.get(entry["IP"])  # reading tuple([urls], [timestamps])
        # comparing last timestamp with new one in seconds
        diff = (entry["DATE"] - t[-1]).total_seconds()
        if diff <= (60 * 30):  # if elapsed time <= 30 minutes
            # we are in the same session for user entry["IP"]
            u.append(urls.index(entry["URL"]))
            t.append(entry["DATE"])
            tmp[entry["IP"]] = (u, t)  # update session for entry["IP"]
        else:
            # a. saving current session from tmp into sessions
            ses = (entry["IP"], u, t)
            sessions[nb_sessions] = ses
            nb_sessions += 1

            # b. starting a new session for user entry["IP"]
            tmp[entry["IP"]] = ([urls.index(entry["URL"])], [entry["DATE"]])

    else:  # building new entry in tmp
        tmp[entry["IP"]] = ([urls.index(entry["URL"])], [entry["DATE"]])

# step 3.3 moving all sessions remaining in tmp to sessions
for key, values in tmp.items():
    sessions[nb_sessions] = (
        key,
        values[0],
        values[1],
    )  # here key = IP, values[0] = [urls], values[1] = [timestamps]
    nb_sessions += 1

# printing some statistics
print("Number of sessions: ", len(sessions), "Number of urls: ", len(urls))


Building sessions and urls dictionary
Number of sessions:  2036 Number of urls:  404


In [7]:
##################################
# Step 4
# Build a dataset where one row = one vector of sessions
# as a numpy array
##################################
import numpy as np

if verbose:
    print("Building Numpy structure with vectorized sessions")

# build the correct sized array
data = np.zeros((len(sessions), len(urls)))

# scan the sessions to build the data matrix
i = 0
for id_user, list_urls, list_timestamps in sessions.values():
    for u in list_urls:  # u values are already indexes of the urls in the list
        data[i][u] += 1
    i += 1

print(data[range(0, 5)])

Building Numpy structure with vectorized sessions
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


In [8]:
##################################
# Step 5
# Analyze the web session by clustering
##################################

if verbose:
    print("Applying clustering approaches")

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# nclusters = 7
for nclusters in range(3, 11):
    kmeans = KMeans(n_clusters=nclusters, random_state=0).fit(data)
    print("Size of discovered clusters: ", np.bincount(kmeans.labels_))
    s = silhouette_score(data, kmeans.labels_)
    print("Quality of clustering with", nclusters, " clusters:", s)

# distance are accessible through:  https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise_distances.html
# clustering algorithms: https://scikit-learn-extra.readthedocs.io/en/latest/generated/sklearn_extra.cluster.KMedoids.html
# to begin with a simple k-means : https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html


Applying clustering approaches
Size of discovered clusters:  [2006   29    1]
Quality of clustering with 3  clusters: 0.8296672877873453
Size of discovered clusters:  [1217  798    1   20]
Quality of clustering with 4  clusters: 0.481721351725549
Size of discovered clusters:  [1215    8    1  771   41]
Quality of clustering with 5  clusters: 0.48766752733448365
Size of discovered clusters:  [  20 1215    8  706    1   86]
Quality of clustering with 6  clusters: 0.48177197116836956
Size of discovered clusters:  [1213    8  170    1  544   80   20]
Quality of clustering with 7  clusters: 0.4661483833067375
Size of discovered clusters:  [ 588    3 1215    1   82   17  125    5]
Quality of clustering with 8  clusters: 0.4737355537584814
Size of discovered clusters:  [   4 1215    1   18  587   78    6  125    2]
Quality of clustering with 9  clusters: 0.4759445947343963
Size of discovered clusters:  [1214   79  586    1    6    4    2    3  124   17]
Quality of clustering with 10  clusters